In [1]:
import pandas as pd 
from pathlib import Path

In [2]:
#dataset_path='../../DRIM/data/TCGA/GBMLGG/raw_RNA/'

In [3]:
dataset_path='../../../data/TCGA/raw/RNA-seq_STAR/'

In [4]:
sample_id ='00362664-cb13-4998-a26c-618dba69ea00'

In [5]:
sample_data_path =Path(dataset_path) / sample_id / 'a7732b3f-6294-47ec-b869-0aac6ee6c815.rna_seq.augmented_star_gene_counts.tsv'

In [6]:
sample_data = pd.read_table(sample_data_path)

In [7]:
sample_data

,,,,,,,,# gene-model: GENCODE v36
gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
N_unmapped,NaN,NaN,2800310,2800310,2800310,NaN,NaN,NaN
N_multimapping,NaN,NaN,6575856,6575856,6575856,NaN,NaN,NaN
N_noFeature,NaN,NaN,4096328,42598823,42961358,NaN,NaN,NaN
N_ambiguous,NaN,NaN,8182106,1964691,1946794,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
ENSG00000288669.1,AC008763.4,protein_coding,0,1,0,0.0000,0.0000,0.0000
ENSG00000288670.1,AL592295.6,lncRNA,1201,632,655,30.1440,9.3454,9.6219
ENSG00000288671.1,AC006486.3,protein_coding,0,0,0,0.0000,0.0000,0.0000
ENSG00000288674.1,AL391628.1,protein_coding,5,2,3,0.0236,0.0073,0.0075


## Воспользуемся препроцессингом из прошлой статьи

In [8]:
# import needed libraries
import pandas as pd
import requests
import io
import pandas as pd
import os
import numpy as np
from collections import OrderedDict
import itertools
from typing import Tuple, Dict
import tqdm
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import StratifiedKFold
from PIL import Image
from IPython import display
# import pyvips
import torch
import torch.nn.functional as F
# import cv2
import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from joblib import dump, load
import torchvision
from torchvision.transforms import ToTensor
import torch
import json

base_path = '../../DRIM/data/TCGA/raw/'

In [9]:
!nvidia-smi

Wed Apr 16 10:15:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:49:00.0 Off |                  Off |
| 30%   30C    P8             26W /  300W |     994MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install opencv-python

In [11]:
dataframe = pd.read_csv('../../DRIM/data/files/clinical_data_all_tcga.tsv', sep='\t')
dataframe.shape

(11081, 17)

In [12]:
dataset_old = pd.read_csv('../../mtcp/src/data/dataset.csv')

In [13]:
dataset_old.group.value_counts()

group
train    885
test     218
Name: count, dtype: int64

In [14]:
dataset_old_test = dataset_old.loc[dataset_old.group =='test']

In [15]:
# dataframe = dataframe[dataframe.project_id.isin(['GBM', 'LGG'])]
# dataframe.shape

In [16]:
dataframe.loc[dataframe['submitter_id'].isin(dataset_old_test['submitter_id']), 'group'] ='test'

In [17]:
dataframe['group'].value_counts()

group
train    8880
test     1202
val       999
Name: count, dtype: int64

In [18]:
dataframe['group'].value_counts()

group
train    8880
test     1202
val       999
Name: count, dtype: int64

In [19]:
dataframe

,submitter_id,tumor_stage,age_at_diagnosis,prior_treatment,prior_malignancy,synchronous_malignancy,days_to_last_follow_up,gender,race,vital_status,days_to_death,treatments_pharmaceutical_treatment_or_therapy,treatments_radiation_treatment_or_therapy,project_id,time,event,group
0,TCGA-3B-A9HI,NaN,24984.0,No,no,No,1521.0,male,white,Alive,NaN,yes,yes,SARC,4.167123,0,val
1,TCGA-3B-A9HJ,NaN,24962.0,No,yes,NaN,1104.0,male,white,Alive,NaN,no,yes,SARC,3.024658,0,train
2,TCGA-3B-A9HL,NaN,24628.0,No,no,No,NaN,male,white,Dead,599.0,yes,no,SARC,1.641096,1,train
3,TCGA-3B-A9HO,NaN,27664.0,Yes,yes,NaN,959.0,male,white,Alive,NaN,no,no,SARC,2.627397,0,train
4,TCGA-3B-A9HP,NaN,21094.0,No,no,No,NaN,female,white,Dead,1627.0,yes,yes,SARC,4.457534,1,val
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11076,TCGA-ZF-AA56,stage iii,28937.0,No,no,No,NaN,female,white,Dead,259.0,no,no,BLCA,0.709589,1,train
11077,TCGA-ZF-AA58,stage iv,22549.0,No,no,No,1649.0,female,white,Alive,NaN,yes,no,BLCA,4.517808,0,train
11078,TCGA-ZF-AA5H,stage iv,22122.0,No,no,No,897.0,female,white,Alive,NaN,yes,no,BLCA,2.457534,0,train
11079,TCGA-ZF-AA5N,stage iv,22741.0,No,no,No,NaN,female,white,Dead,168.0,yes,no,BLCA,0.460274,1,train


In [20]:
# helpers function
def request_file_info(data_type):
    """
    Adapted from Vale-Silva's preprocessing.
    """
    fields = [
        'file_name',
        'cases.submitter_id',
        'cases.samples.sample_type',
        'cases.project.project_id',
        'cases.project.primary_site',
        ]

    fields = ','.join(fields)

    files_endpt = 'https://api.gdc.cancer.gov/files'

    filters = {
        'op': 'and',
        'content':[
             # {
        # "op": "in",
        # "content":{
        #     "field": "cases.project.primary_site",
        #     "value": ["Brain"]
        #     }
        # },
            {
            'op': 'in',
            'content':{
                'field': 'files.experimental_strategy',
                'value': [data_type]
                }
            }  
        ]
    }

    params = {
        'filters': filters,
        'fields': fields,
        'format': 'TSV',
        'size': '1000000'
        }

    response = requests.post(files_endpt, headers = {'Content-Type': 'application/json'}, json = params)

    return pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep='\t')

In [21]:
def make_patient_file_map(df, base_dir):
    """
    Adapted from Vale-Silva's preprocessing.
    """
    
    d = {}
    for _, row in df.iterrows():
        patient = row['cases.0.submitter_id']
        if os.path.exists(os.path.join(base_dir, row.id, row.file_name)):
            if patient in d:
                if not isinstance(d[patient], tuple):
                    d[patient] = (
                        d[patient],
                        os.path.join(base_dir, row.id, row.file_name))
                else:
                    d[patient] += os.path.join(base_dir, row.id, row.file_name),
            else:
                d[patient] = os.path.join(base_dir, row.id, row.file_name)

    return d

# 2. Clinical Data <a name="p2"></a>

In [22]:
# %%capture
# supplementary = pd.read_csv('../../DRIM/data/files/supplementary_data.tsv', sep='\t')

In [23]:
# mapper_clin = {
#     'patient': 'submitter_id', 
#     'IDH.status': 'IDH', 
#     'MGMT.promoter.status': 'MGMT',
#     'X1p.19q.codeletion': 'X1p19q',
#     'Chr.19.20.co.gain': '19.20.gain',
#     'Chr.7.gain.Chr.10.loss': '7g10l',
#     'TERT.expression.status': 'TERT',
#     'ATRX.status': 'ATRX',
# }

In [24]:
# supplementary = supplementary.drop(columns=['IDH', 'Mutations', 'TERT']).rename(columns=mapper_clin)[list(mapper_clin.values())]
# supplementary = supplementary[~supplementary.duplicated(subset=['submitter_id'], keep='first')].dropna(subset='submitter_id')

In [25]:
# dataframe = dataframe.merge(supplementary, how='left', on='submitter_id')
# dataframe.shape

In [26]:
# for col in list(mapper_clin.values()):
#     print(dataframe[col].value_counts())

In [27]:
dataframe

,submitter_id,tumor_stage,age_at_diagnosis,prior_treatment,prior_malignancy,synchronous_malignancy,days_to_last_follow_up,gender,race,vital_status,days_to_death,treatments_pharmaceutical_treatment_or_therapy,treatments_radiation_treatment_or_therapy,project_id,time,event,group
0,TCGA-3B-A9HI,NaN,24984.0,No,no,No,1521.0,male,white,Alive,NaN,yes,yes,SARC,4.167123,0,val
1,TCGA-3B-A9HJ,NaN,24962.0,No,yes,NaN,1104.0,male,white,Alive,NaN,no,yes,SARC,3.024658,0,train
2,TCGA-3B-A9HL,NaN,24628.0,No,no,No,NaN,male,white,Dead,599.0,yes,no,SARC,1.641096,1,train
3,TCGA-3B-A9HO,NaN,27664.0,Yes,yes,NaN,959.0,male,white,Alive,NaN,no,no,SARC,2.627397,0,train
4,TCGA-3B-A9HP,NaN,21094.0,No,no,No,NaN,female,white,Dead,1627.0,yes,yes,SARC,4.457534,1,val
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11076,TCGA-ZF-AA56,stage iii,28937.0,No,no,No,NaN,female,white,Dead,259.0,no,no,BLCA,0.709589,1,train
11077,TCGA-ZF-AA58,stage iv,22549.0,No,no,No,1649.0,female,white,Alive,NaN,yes,no,BLCA,4.517808,0,train
11078,TCGA-ZF-AA5H,stage iv,22122.0,No,no,No,897.0,female,white,Alive,NaN,yes,no,BLCA,2.457534,0,train
11079,TCGA-ZF-AA5N,stage iv,22741.0,No,no,No,NaN,female,white,Dead,168.0,yes,no,BLCA,0.460274,1,train


# 4. RNASeq <a name="p4"></a>

In [28]:
%%time

mRNA_files = request_file_info(data_type='RNA-Seq')
mRNA_files.shape

CPU times: user 894 ms, sys: 283 ms, total: 1.18 s
Wall time: 1min 12s


/tmp/ipykernel_808/4195230446.py:47: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep='\t')


(223002, 10)

In [29]:
mRNA_files = mRNA_files[mRNA_files['cases.0.project.project_id'].str.startswith('TCGA')]

In [30]:
mRNA_files[mRNA_files['file_name'].str.endswith('counts.tsv')]['file_name'].str[-10:]

373       counts.tsv
377       counts.tsv
385       counts.tsv
395       counts.tsv
408       counts.tsv
             ...    
222977    counts.tsv
222983    counts.tsv
222989    counts.tsv
222994    counts.tsv
222997    counts.tsv
Name: file_name, Length: 11499, dtype: object

In [31]:
mRNA_files = mRNA_files[mRNA_files['cases.0.project.project_id'].str.startswith('TCGA')]
mRNA_files = mRNA_files[mRNA_files['file_name'].str.endswith('rna_seq.augmented_star_gene_counts.tsv')]
mRNA_files = mRNA_files[mRNA_files['cases.0.samples.0.sample_type'] == 'Primary Tumor']
mRNA_files.shape

(10151, 10)

In [32]:
print('All rows:       ', mRNA_files.shape[0])
print('Unique patients:', mRNA_files['cases.0.submitter_id'].unique().shape[0])

All rows:        10151
Unique patients: 9965


In [33]:
mRNA_files = mRNA_files[~mRNA_files.duplicated(subset=['cases.0.submitter_id'], keep='first')]
mRNA_files.shape

(9965, 10)

In [34]:
len(mRNA_files['cases.0.submitter_id'].unique())

9965

In [35]:
base_path = '../../DRIM/data/TCGA/raw/'

In [36]:
base_path ='../../../data/TCGA/raw/'

In [37]:
len(os.listdir(os.path.join(base_path, 'RNA-seq_STAR')))

10302

In [38]:
mRNA_files

,cases.0.project.primary_site,cases.0.project.project_id,cases.0.samples.0.sample_type,cases.0.samples.1.sample_type,cases.0.samples.2.sample_type,cases.0.samples.3.sample_type,cases.0.samples.4.sample_type,cases.0.submitter_id,file_name,id
377,Head and Neck,TCGA-HNSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-CV-7423,f4f442e1-b6cf-467a-ae5e-ccf28a7f940b.rna_seq.a...,568f568a-405a-4b79-a52b-a48b712cb4fd
385,Head and Neck,TCGA-HNSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-DQ-7595,bdbe42b1-9e8a-408e-ad13-940f186618ef.rna_seq.a...,d1ba03b4-c63c-4be6-a723-6d87749e4b61
395,Head and Neck,TCGA-HNSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-CR-7377,1ee4cd4e-7386-493a-9a36-421856a2b924.rna_seq.a...,efbbaf9c-6f7c-4a67-a741-d47133001cd3
408,Head and Neck,TCGA-HNSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-CR-7371,44d6fd71-0853-48e6-82c8-c4c1978278aa.rna_seq.a...,50fd12e8-71e9-40e9-8752-33cc2b537000
409,Head and Neck,TCGA-HNSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-T2-A6X0,59f36c25-d612-4fa8-a52e-7d910f6ed8d5.rna_seq.a...,7e910e81-71a8-4120-993e-746d29902026
...,...,...,...,...,...,...,...,...,...,...
222977,Lung,TCGA-LUSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-39-5028,77b0bb2f-6e9f-4c67-b64c-98eebf0bd99d.rna_seq.a...,a2ff795f-2f9f-4a67-a8e8-5d1b70f397d8
222983,Lung,TCGA-LUSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-NC-A5HE,4f179a2f-1e37-4ad9-9f21-0a6f1ecdf03e.rna_seq.a...,5bb8ce26-770f-48cc-a481-466469d34073
222989,Lung,TCGA-LUSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-66-2783,3c174eac-9153-4c66-8da4-969925c2c4c4.rna_seq.a...,c8a68cbe-87a1-44b3-bf4b-68ba0de91994
222994,Lung,TCGA-LUSC,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-66-2795,5332a617-f178-4197-8ed4-2f49bf8751e0.rna_seq.a...,5c3ec03d-d9eb-48e8-b132-5733dcf83131


In [39]:
file_map = make_patient_file_map(mRNA_files, os.path.join(base_path, 'RNA-seq_STAR'))
file_map = {k: file_map[k] for k in file_map if k in list(dataframe['submitter_id'])}
len(file_map)

9889

In [40]:
RNA_mapping = pd.DataFrame([(k, v) for k, v in file_map.items()], columns=('submitter_id', 'RNA'))
dataframe = dataframe.merge(RNA_mapping, how='left', on='submitter_id')

In [41]:
dataframe

,submitter_id,tumor_stage,age_at_diagnosis,prior_treatment,prior_malignancy,synchronous_malignancy,days_to_last_follow_up,gender,race,vital_status,days_to_death,treatments_pharmaceutical_treatment_or_therapy,treatments_radiation_treatment_or_therapy,project_id,time,event,group,RNA
0,TCGA-3B-A9HI,NaN,24984.0,No,no,No,1521.0,male,white,Alive,NaN,yes,yes,SARC,4.167123,0,val,../../../data/TCGA/raw/RNA-seq_STAR/6e2d183b-d...
1,TCGA-3B-A9HJ,NaN,24962.0,No,yes,NaN,1104.0,male,white,Alive,NaN,no,yes,SARC,3.024658,0,train,../../../data/TCGA/raw/RNA-seq_STAR/d1ad0704-a...
2,TCGA-3B-A9HL,NaN,24628.0,No,no,No,NaN,male,white,Dead,599.0,yes,no,SARC,1.641096,1,train,../../../data/TCGA/raw/RNA-seq_STAR/48bddd13-9...
3,TCGA-3B-A9HO,NaN,27664.0,Yes,yes,NaN,959.0,male,white,Alive,NaN,no,no,SARC,2.627397,0,train,../../../data/TCGA/raw/RNA-seq_STAR/023c4fe4-9...
4,TCGA-3B-A9HP,NaN,21094.0,No,no,No,NaN,female,white,Dead,1627.0,yes,yes,SARC,4.457534,1,val,../../../data/TCGA/raw/RNA-seq_STAR/3f907fca-d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11076,TCGA-ZF-AA56,stage iii,28937.0,No,no,No,NaN,female,white,Dead,259.0,no,no,BLCA,0.709589,1,train,../../../data/TCGA/raw/RNA-seq_STAR/11528963-2...
11077,TCGA-ZF-AA58,stage iv,22549.0,No,no,No,1649.0,female,white,Alive,NaN,yes,no,BLCA,4.517808,0,train,../../../data/TCGA/raw/RNA-seq_STAR/ab30d269-4...
11078,TCGA-ZF-AA5H,stage iv,22122.0,No,no,No,897.0,female,white,Alive,NaN,yes,no,BLCA,2.457534,0,train,../../../data/TCGA/raw/RNA-seq_STAR/545e71df-0...
11079,TCGA-ZF-AA5N,stage iv,22741.0,No,no,No,NaN,female,white,Dead,168.0,yes,no,BLCA,0.460274,1,train,../../../data/TCGA/raw/RNA-seq_STAR/90fb6b38-0...


In [42]:
dataframe.RNA.notnull().sum()

9889

## Preprocessing
In this section, we only consider genes coding for proteins, and we also remove genes with a variance of less than 0.1 on the training set.

In [43]:
def load_single_file_RNA(path):
    temp_df = pd.read_csv(path, sep='\t', skiprows=6, usecols=[0, 2, 8], names=['gene_id', 'gene_type', 'fpkm_uq_unstranded']).set_index('gene_id')#, 2, 3, 4, 5, 6, 7, 8])
    temp_df = temp_df[temp_df['gene_type'] == 'protein_coding']['fpkm_uq_unstranded']
    return temp_df

dataframe_train = dataframe[dataframe.group == 'train']
file_map_train = make_patient_file_map(mRNA_files, os.path.join(base_path, 'raw_RNA'))
file_map_train = {k: file_map[k] for k in file_map if k in list(dataframe_train['submitter_id'])}
len(file_map_train)

7922

In [44]:
rna_train_dataframe = pd.concat([load_single_file_RNA(path) for path in file_map_train.values()], axis=1)

In [45]:
gene_description = rna_train_dataframe.T.describe()

In [46]:
gene_description

gene_id,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288649.1,ENSG00000288654.1,ENSG00000288656.1,ENSG00000288658.1,ENSG00000288660.1,ENSG00000288661.1,ENSG00000288669.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
count,7922.000000,7922.000000,7922.000000,7922.000000,7922.000000,7922.000000,7922.000000,7922.000000,7922.000000,7922.000000,...,7922.0,7922.0,7922.000000,7922.000000,7922.000000,7922.0,7922.000000,7922.0,7922.000000,7922.000000
mean,17.852449,1.158213,34.702865,2.437499,1.605907,3.879637,19.272030,21.123642,8.219360,10.351091,...,0.0,0.0,0.004943,0.423401,0.332653,0.0,0.001811,0.0,0.015038,0.423062
std,15.058335,24.161319,18.063895,1.262713,1.243282,4.623268,62.284528,13.100190,13.199865,5.628819,...,0.0,0.0,0.042957,1.287883,1.445694,0.0,0.017770,0.0,0.025391,0.435134
min,0.137700,0.000000,1.274700,0.153400,0.012800,0.017900,0.050600,0.333800,0.138300,1.809000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
25%,9.024500,0.000000,23.339275,1.652925,0.730125,1.401125,2.191875,13.106075,3.373450,7.104800,...,0.0,0.0,0.000000,0.029700,0.000000,0.0,0.000000,0.0,0.006100,0.189000
50%,13.950550,0.040800,30.072250,2.155750,1.325350,2.558400,5.563500,18.692950,4.942500,9.255600,...,0.0,0.0,0.000000,0.104500,0.000000,0.0,0.000000,0.0,0.011500,0.302300
75%,21.399850,0.167900,41.225225,2.839900,2.125250,4.751725,13.027500,26.337300,7.658125,12.161325,...,0.0,0.0,0.005300,0.354050,0.061750,0.0,0.000000,0.0,0.019500,0.507875
max,204.289800,1590.466100,267.599700,17.912500,21.551400,82.104200,1019.497200,273.103600,293.536400,170.303600,...,0.0,0.0,3.557600,33.965600,27.776100,0.0,1.311500,0.0,1.875300,8.786800


In [47]:
genes_to_keep = gene_description.columns[gene_description.iloc[2, :] > 0.1].values.tolist()

In [48]:
def load_preprocessed_single_file_RNA(path, genes_to_keep=genes_to_keep):
    temp_df = load_single_file_RNA(path)
    return temp_df.loc[genes_to_keep]

In [49]:
len(genes_to_keep)

18244

In [50]:
#base_path = '../../DRIM/data/TCGA/raw/'

In [51]:
base_path ='../../../data/TCGA/raw/'

In [52]:
output_path = os.path.join(base_path, 'RNA')

if not os.path.exists(output_path):
    os.makedirs(output_path)

for patient, filepath in tqdm.tqdm(file_map.items()):
    idx, filename = filepath.split('/')[-2:]
    if not os.path.exists(os.path.join(output_path, idx)):
        os.makedirs(os.path.join(output_path, idx))
        
    temp_df = load_preprocessed_single_file_RNA(filepath)
    temp_df.to_csv(os.path.join(output_path, idx, filename.replace('tsv', 'csv')))

100%|███████████████████████████████████████| 9889/9889 [17:18<00:00,  9.52it/s]


In [53]:
def replace_rna(x):
    if pd.isna(x):
        return x
    else:
        return str(x).replace('raw_RNA', 'RNA').replace('tsv', 'csv')
    
    
dataframe['RNA'] = dataframe['RNA'].apply(replace_rna)

In [54]:
dataframe.to_csv('../../DRIM/data/files/dataset_full_tcga.csv', index=False)

In [55]:
dataframe.to_csv('../../mtcp/src/data/dataset_full_tcga.csv', index=False)

In [56]:
len(os.listdir(os.path.join('../../../data/TCGA/raw/', 'RNA-seq_STAR')))

10302

In [57]:
dataframe

,submitter_id,tumor_stage,age_at_diagnosis,prior_treatment,prior_malignancy,synchronous_malignancy,days_to_last_follow_up,gender,race,vital_status,days_to_death,treatments_pharmaceutical_treatment_or_therapy,treatments_radiation_treatment_or_therapy,project_id,time,event,group,RNA
0,TCGA-3B-A9HI,NaN,24984.0,No,no,No,1521.0,male,white,Alive,NaN,yes,yes,SARC,4.167123,0,val,../../../data/TCGA/raw/RNA-seq_STAR/6e2d183b-d...
1,TCGA-3B-A9HJ,NaN,24962.0,No,yes,NaN,1104.0,male,white,Alive,NaN,no,yes,SARC,3.024658,0,train,../../../data/TCGA/raw/RNA-seq_STAR/d1ad0704-a...
2,TCGA-3B-A9HL,NaN,24628.0,No,no,No,NaN,male,white,Dead,599.0,yes,no,SARC,1.641096,1,train,../../../data/TCGA/raw/RNA-seq_STAR/48bddd13-9...
3,TCGA-3B-A9HO,NaN,27664.0,Yes,yes,NaN,959.0,male,white,Alive,NaN,no,no,SARC,2.627397,0,train,../../../data/TCGA/raw/RNA-seq_STAR/023c4fe4-9...
4,TCGA-3B-A9HP,NaN,21094.0,No,no,No,NaN,female,white,Dead,1627.0,yes,yes,SARC,4.457534,1,val,../../../data/TCGA/raw/RNA-seq_STAR/3f907fca-d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11076,TCGA-ZF-AA56,stage iii,28937.0,No,no,No,NaN,female,white,Dead,259.0,no,no,BLCA,0.709589,1,train,../../../data/TCGA/raw/RNA-seq_STAR/11528963-2...
11077,TCGA-ZF-AA58,stage iv,22549.0,No,no,No,1649.0,female,white,Alive,NaN,yes,no,BLCA,4.517808,0,train,../../../data/TCGA/raw/RNA-seq_STAR/ab30d269-4...
11078,TCGA-ZF-AA5H,stage iv,22122.0,No,no,No,897.0,female,white,Alive,NaN,yes,no,BLCA,2.457534,0,train,../../../data/TCGA/raw/RNA-seq_STAR/545e71df-0...
11079,TCGA-ZF-AA5N,stage iv,22741.0,No,no,No,NaN,female,white,Dead,168.0,yes,no,BLCA,0.460274,1,train,../../../data/TCGA/raw/RNA-seq_STAR/90fb6b38-0...


In [58]:
train_brain = dataframe[dataframe.group == 'train']
test_brain = dataframe[dataframe.group == 'test']

In [59]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
splits = np.zeros(len(train_brain))
for i, (_, fold_idx) in enumerate(skf.split(train_brain, train_brain.project_id)):
    splits[fold_idx] = i

In [60]:
train_brain['splits'] =splits

/tmp/ipykernel_808/1158904737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_brain['splits'] =splits


In [61]:
train_brain

,submitter_id,tumor_stage,age_at_diagnosis,prior_treatment,prior_malignancy,synchronous_malignancy,days_to_last_follow_up,gender,race,vital_status,days_to_death,treatments_pharmaceutical_treatment_or_therapy,treatments_radiation_treatment_or_therapy,project_id,time,event,group,RNA,splits
1,TCGA-3B-A9HJ,NaN,24962.0,No,yes,NaN,1104.0,male,white,Alive,NaN,no,yes,SARC,3.024658,0,train,../../../data/TCGA/raw/RNA-seq_STAR/d1ad0704-a...,0.0
2,TCGA-3B-A9HL,NaN,24628.0,No,no,No,NaN,male,white,Dead,599.0,yes,no,SARC,1.641096,1,train,../../../data/TCGA/raw/RNA-seq_STAR/48bddd13-9...,4.0
3,TCGA-3B-A9HO,NaN,27664.0,Yes,yes,NaN,959.0,male,white,Alive,NaN,no,no,SARC,2.627397,0,train,../../../data/TCGA/raw/RNA-seq_STAR/023c4fe4-9...,2.0
5,TCGA-3B-A9HQ,NaN,24386.0,No,no,No,2085.0,female,white,Alive,NaN,no,no,SARC,5.712329,0,train,../../../data/TCGA/raw/RNA-seq_STAR/bb540c2d-1...,0.0
6,TCGA-3B-A9HR,NaN,14041.0,No,no,No,NaN,female,black or african american,Dead,2694.0,yes,yes,SARC,7.380822,1,train,../../../data/TCGA/raw/RNA-seq_STAR/39d68145-7...,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11076,TCGA-ZF-AA56,stage iii,28937.0,No,no,No,NaN,female,white,Dead,259.0,no,no,BLCA,0.709589,1,train,../../../data/TCGA/raw/RNA-seq_STAR/11528963-2...,1.0
11077,TCGA-ZF-AA58,stage iv,22549.0,No,no,No,1649.0,female,white,Alive,NaN,yes,no,BLCA,4.517808,0,train,../../../data/TCGA/raw/RNA-seq_STAR/ab30d269-4...,1.0
11078,TCGA-ZF-AA5H,stage iv,22122.0,No,no,No,897.0,female,white,Alive,NaN,yes,no,BLCA,2.457534,0,train,../../../data/TCGA/raw/RNA-seq_STAR/545e71df-0...,1.0
11079,TCGA-ZF-AA5N,stage iv,22741.0,No,no,No,NaN,female,white,Dead,168.0,yes,no,BLCA,0.460274,1,train,../../../data/TCGA/raw/RNA-seq_STAR/90fb6b38-0...,3.0


In [62]:
train_brain.loc[train_brain['RNA'].notnull()]['splits'].value_counts()

splits
1.0    1592
2.0    1588
0.0    1586
3.0    1586
4.0    1570
Name: count, dtype: int64

In [63]:
pd.concat([train_brain, test_brain]).to_csv("../src/data/dataset_full_tcga.csv", index=False)

In [64]:
gbmlgg = dataframe.loc[dataframe['project_id'].isin(['GBM', 'LGG'])]

In [71]:
gbmlgg.shape

(1104, 18)

In [65]:
gbmlgg.loc[gbmlgg['RNA'].notnull()].submitter_id.value_counts()

submitter_id
TCGA-CS-4938    1
TCGA-02-0070    1
TCGA-02-2470    1
TCGA-02-2483    1
TCGA-02-2485    1
               ..
TCGA-HT-7689    1
TCGA-HT-7690    1
TCGA-HT-7691    1
TCGA-HT-7692    1
TCGA-76-4932    1
Name: count, Length: 792, dtype: int64

In [72]:
gbmlgg.loc[gbmlgg['RNA'].notnull()].shape

(792, 18)

In [67]:
dataset_path2 ='../src/data/dataset.csv'

In [68]:
dataset2=pd.read_csv(dataset_path2)

In [69]:
gene_ids = pd.read_csv("../../" + dataset2.iloc[0, :]['RNA']).gene_id.values.tolist()

In [70]:
len(genes_to_keep)

18244

In [98]:
len(gene_ids)

16304

In [99]:
len(set(gene_ids) & set(genes_to_keep))

16264

In [100]:
genes_to_keep = gene_ids

In [107]:
mRNA_files = request_file_info(data_type='RNA-Seq')
mRNA_files.shape

/tmp/ipykernel_713/4195230446.py:47: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep='\t')


(223002, 10)

In [108]:
mRNA_files =mRNA_files.loc[mRNA_files["cases.0.project.project_id"].isin(['TCGA-LGG', 'TCGA-GBM'])]

In [109]:
mRNA_files = mRNA_files[mRNA_files['file_name'].str.endswith('rna_seq.augmented_star_gene_counts.tsv')]

In [110]:
mRNA_files.loc[mRNA_files['cases.0.submitter_id']=='TCGA-06-0681']

,cases.0.project.primary_site,cases.0.project.project_id,cases.0.samples.0.sample_type,cases.0.samples.1.sample_type,cases.0.samples.2.sample_type,cases.0.samples.3.sample_type,cases.0.samples.4.sample_type,cases.0.submitter_id,file_name,id
215452,Brain,TCGA-GBM,Solid Tissue Normal,NaN,NaN,NaN,NaN,TCGA-06-0681,4cc96a03-4717-465b-b0c6-dfb947ff3c5e.rna_seq.a...,c410d37e-e445-4f5f-8937-e6796dd52364


In [113]:
mRNA_files.loc[mRNA_files['cases.0.submitter_id']=='TCGA-06-0190']

,cases.0.project.primary_site,cases.0.project.project_id,cases.0.samples.0.sample_type,cases.0.samples.1.sample_type,cases.0.samples.2.sample_type,cases.0.samples.3.sample_type,cases.0.samples.4.sample_type,cases.0.submitter_id,file_name,id
216791,Brain,TCGA-GBM,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-06-0190,59f436d5-7a66-40a8-9bca-b1e354452492.rna_seq.a...,27ee0eba-2f11-447b-a153-c8dc679d5fc6
216796,Brain,TCGA-GBM,Recurrent Tumor,NaN,NaN,NaN,NaN,TCGA-06-0190,2c25aa49-879a-4023-80b4-84df717dc537.rna_seq.a...,9916e085-1027-4c91-9729-93dc36b71c86
216801,Brain,TCGA-GBM,Primary Tumor,NaN,NaN,NaN,NaN,TCGA-06-0190,dad61e18-e3f1-4beb-b3c3-ae434e35af2d.rna_seq.a...,ba9afa8e-405c-49b7-bc66-8ab1af1f942e


In [111]:
mRNA_files['cases.0.submitter_id'].value_counts()

cases.0.submitter_id
TCGA-06-0211    4
TCGA-06-0190    3
TCGA-19-4065    3
TCGA-06-0210    3
TCGA-TQ-A7RK    3
               ..
TCGA-TM-A84C    1
TCGA-HT-7884    1
TCGA-FG-A87N    1
TCGA-TM-A84O    1
TCGA-12-3646    1
Name: count, Length: 809, dtype: int64

In [117]:
(mRNA_files['cases.0.submitter_id'].value_counts()>1).sum()

104

In [100]:
mRNA_files['cases.0.samples.0.sample_type'].value_counts()

cases.0.samples.0.sample_type
Primary Tumor          888
Recurrent Tumor         32
Solid Tissue Normal      5
Name: count, dtype: int64

In [101]:
mRNA_files =mRNA_files.loc[~mRNA_files.duplicated(subset=['cases.0.submitter_id'], keep='first')]

In [106]:
mRNA_files.loc[mRNA_files['cases.0.samples.0.sample_type']=='Solid Tissue Normal']

,cases.0.project.primary_site,cases.0.project.project_id,cases.0.samples.0.sample_type,cases.0.samples.1.sample_type,cases.0.samples.2.sample_type,cases.0.samples.3.sample_type,cases.0.samples.4.sample_type,cases.0.submitter_id,file_name,id
215452,Brain,TCGA-GBM,Solid Tissue Normal,NaN,NaN,NaN,NaN,TCGA-06-0681,4cc96a03-4717-465b-b0c6-dfb947ff3c5e.rna_seq.a...,c410d37e-e445-4f5f-8937-e6796dd52364
216128,Brain,TCGA-GBM,Solid Tissue Normal,NaN,NaN,NaN,NaN,TCGA-06-0675,2856d609-b09a-486e-8552-23abaa1df201.rna_seq.a...,4bed9101-07f4-4d76-b79f-6eb8de04bd19
216544,Brain,TCGA-GBM,Solid Tissue Normal,NaN,NaN,NaN,NaN,TCGA-06-0678,152343f6-832e-4077-83d3-1589ef71b472.rna_seq.a...,ddaed459-ab19-4f75-a1a9-72761287c6f5
216696,Brain,TCGA-GBM,Solid Tissue Normal,NaN,NaN,NaN,NaN,TCGA-06-0680,3404cbdb-4d1b-462a-a820-58c4f01e84fc.rna_seq.a...,64a2ddbe-3614-409a-af16-170a1471db2a
217362,Brain,TCGA-GBM,Solid Tissue Normal,NaN,NaN,NaN,NaN,TCGA-06-AABW,4250878d-ba0b-47e1-96c6-f2331f93ce37.rna_seq.a...,477a4ae1-84c0-49f2-b3b3-4015b8c26f18


In [103]:
mRNA_files['cases.0.samples.0.sample_type'].value_counts()

cases.0.samples.0.sample_type
Primary Tumor          788
Recurrent Tumor         16
Solid Tissue Normal      5
Name: count, dtype: int64

In [104]:
file_map = make_patient_file_map(mRNA_files, os.path.join(base_path, 'RNA-seq_STAR'))
file_map = {k: file_map[k] for k in file_map if k in list(dataframe['submitter_id'])}
len(file_map)

780

In [105]:
mRNA_files['cases.0.project.primary_site'].value_counts()

cases.0.project.primary_site
Brain    809
Name: count, dtype: int64